# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [3]:
import json

with open('contributors_sample.json', 'r') as f:
    data = json.load(f)

# вывод информации о первых трёх пользователях
for user in data[:3]:
    print("User username:", user['username'])
    print("User name:", user["name"])
    print("User address:", user["address"])
    print("User mail:", user["mail"])
    print("User sex:", user["sex"])
    print("---")

User username: uhebert
User name: Lindsey Nguyen
User address: 01261 Cameron Spring
Taylorfurt, AK 97791
User mail: jsalazar@gmail.com
User sex: F
---
User username: vickitaylor
User name: Cheryl Lewis
User address: 66992 Welch Brooks
Marshallshire, ID 56004
User mail: bhudson@gmail.com
User sex: F
---
User username: sheilaadams
User name: Julia Allen
User address: Unit 1632 Box 2971
DPO AE 23297
User mail: darren44@yahoo.com
User sex: F
---


In [11]:
import pandas as pd

df = pd.read_json('contributors_sample.json')

df.head(3)

,username,name,sex,address,mail,jobs,id
0,uhebert,Lindsey Nguyen,F,"01261 Cameron Spring\nTaylorfurt, AK 97791",jsalazar@gmail.com,"[Energy engineer, Engineer, site, Environmenta...",35193
1,vickitaylor,Cheryl Lewis,F,"66992 Welch Brooks\nMarshallshire, ID 56004",bhudson@gmail.com,"[Music therapist, Volunteer coordinator, Desig...",91970
2,sheilaadams,Julia Allen,F,Unit 1632 Box 2971\nDPO AE 23297,darren44@yahoo.com,"[Management consultant, Engineer, structural, ...",1848091


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [12]:
import pandas as pd

df['Domain'] = df['mail'].str.split('@').str[1]

unique_domains = df['Domain'].unique()

unique_domains


array(['gmail.com', 'yahoo.com', 'hotmail.com'], dtype=object)

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [18]:
def find_user(username):
    if df[df['username'] == username].shape[0]==0:
        raise ValueError()
    else:
        return df[df['username']==username]
find_user('jean67')

,username,name,sex,address,mail,jobs,id,Domain
4,jean67,Nicholas Harrington,M,9080 Monica Crescent Suite 820\nNorth Deanbury...,denise42@gmail.com,"[Network engineer, Youth worker, Primary schoo...",676820,gmail.com


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [20]:
df1 = df.groupby(['sex']).count()
df1

,username,name,address,mail,jobs,id,Domain
sex,,,,,,,
F,2136,2136,2136,2136,2136,2136,2136
M,2064,2064,2064,2064,2064,2064,2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [25]:
contributors = df[['id', 'username', 'sex']]
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [26]:
recipes = pd.read_csv('recipes_sample.csv')
merged_df = pd.merge(recipes, contributors, on='id', how='left')
missing_contributors = merged_df[merged_df['username'].isnull()]
num_missing = len(missing_contributors)
print(f"Информация отсутствует для {num_missing} человек")

Информация отсутствует для 29830 человек


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [32]:
import json

with open('contributors_sample.json') as f:
    contributors = json.load(f)
positions = {}

for person in contributors:
    if 'position' in person:
        position = person['position']
        username = person['username']
        if position not in positions:
            positions[position] = [username]
        else:
            positions[position].append(username)
print(positions)

{}


2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`